## AlphabetSoupCharity Run 2
In this notebook we will attempt to reduce the features that may not be important to the prediction.  Identify any additional columns that are not likely to affect the success prediction.

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

## Investigate other features to see if they might be useless features

In [4]:
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [5]:
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [6]:
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [7]:
# Drop ORGANIZATION and AFFILIATION.  Might just be that it doesn't matter what your charity origin category is to have a successful event
application_df = application_df.drop(['ORGANIZATION', 'AFFILIATION'], axis=1)
application_df

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,1,0,N,5000,1
1,T3,C2000,Preservation,1,1-9999,N,108590,1
2,T5,C3000,ProductDev,1,0,N,5000,0
3,T3,C2000,Preservation,1,10000-24999,N,6692,1
4,T3,C1000,Heathcare,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,C1000,ProductDev,1,0,N,5000,0
34295,T4,C3000,ProductDev,1,0,N,5000,0
34296,T3,C2000,Preservation,1,0,N,5000,0
34297,T5,C3000,ProductDev,1,0,N,5000,1


In [8]:
# Choose a cutoff value and create a list of application types (binning)
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications (binning)
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Verify all of the data types in the dataframe
application_df.dtypes

APPLICATION_TYPE          object
CLASSIFICATION            object
USE_CASE                  object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [11]:
# Feature SPECIAL_CONSIDERATIONS has only 2 values.  Use label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
application_df["SPECIAL_CONSIDERATIONS"] = le.fit_transform(application_df["SPECIAL_CONSIDERATIONS"])
print("Label encoder for SPECIAL_CONSIDERATIONS: ", le.classes_)
application_df.head()

Label encoder for SPECIAL_CONSIDERATIONS:  ['N' 'Y']


,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,1,0,0,5000,1
1,T3,C2000,Preservation,1,1-9999,0,108590,1
2,T5,C3000,ProductDev,1,0,0,5000,0
3,T3,C2000,Preservation,1,10000-24999,0,6692,1
4,T3,C1000,Heathcare,1,100000-499999,0,142590,1


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
34295,1,0,5000,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
34296,1,0,5000,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,5000,1,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,0,0


In [13]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop('IS_SUCCESSFUL', axis=1)
y = application_dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model (32 features instead of 42 from original run)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Input_dimL1 is 42 (not 43) because feature SPECIAL_CONSIDERATIONS remained as one feature with label-encoder.
unitsL1=80
activationL1="relu"
input_dimL1=32

unitsL2=30
activationL2="relu"


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=unitsL1, activation=activationL1, input_dim=input_dimL1))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=unitsL2, activation=activationL2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2640      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
HDF5_Save = '../HDF5/AlphabetSoupCharity_Run2.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(HDF5_Save)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_cb])

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6416 - accuracy: 0.6155
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6343 - accuracy: 0.6239
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6326 - accuracy: 0.6283
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6320 - accuracy: 0.6285
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6309 - accuracy: 0.6261
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6305 - accuracy: 0.6285
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6301 - accuracy: 0.6294
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6299 - accuracy: 0.6285
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6295 - accuracy: 0.6287
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6290 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6362 - accuracy: 0.6253 - 356ms/epoch - 1ms/step
Loss: 0.6362357139587402, Accuracy: 0.6253061294555664


In [19]:
# Export our model to HDF5 file
nn.save(HDF5_Save)